In [1]:
import logging
logging.getLogger("pdfminer").setLevel(logging.ERROR)

import os
import glob
import pdfplumber

pdf_dir = "../data/input/pdf/"

# チャンク化関数（1ファイル分）
def chunk_pdf(file_path):
    chunks = []
    with pdfplumber.open(file_path) as pdf:
        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            if text:
                chunks.append({
                    "chunk_id": f"{file_path}_page_{i+1}",
                    "text": text.strip(),
                    "page": i + 1,
                    "file_path": file_path,
                })
    return chunks

# すべてのPDFを処理してまとめてチャンク化
all_chunks = []
pdf_files = glob.glob(os.path.join(pdf_dir, "*.pdf"))

print(f"PDFファイル数: {len(pdf_files)}")

for file_path in pdf_files:
    chunks = chunk_pdf(file_path)
    all_chunks.extend(chunks)
    print(f"{os.path.basename(file_path)}: {len(chunks)} チャンク")

print(f"\n総チャンク数: {len(all_chunks)}")

if all_chunks:
    print(f"\n📄 page {all_chunks[0]['page']} ({all_chunks[0]['file_path']}) の内容:\n")
    print(all_chunks[0]["text"][:100])


PDFファイル数: 12
20250424_4release_jp.pdf: 3 チャンク
tenpo_all.pdf: 20 チャンク
20250307release_jp.pdf: 2 チャンク
20240220release_jp.pdf: 3 チャンク
mmc2020.pdf: 12 チャンク
news130711.pdf: 1 チャンク
news140331_2.pdf: 1 チャンク
news131119.pdf: 1 チャンク
20250328_4release_jp.pdf: 2 チャンク
20250114release_jp.pdf: 1 チャンク
20250129release_jp.pdf: 1 チャンク
tenpo_01.pdf: 7 チャンク

総チャンク数: 54

📄 page 1 (../data/input/pdf/20250424_4release_jp.pdf) の内容:

2025 年 4 月 24 日
株式会社みずほ銀行
GMO イプシロン株式会社
企業間決済プラットフォーム「M's PayBridge」の取扱開始について
～業務プロセスと決済のシームレスな融合～
株


In [2]:
import pandas as pd

df = pd.DataFrame(chunks)
df[["page", "text"]].head(5)


,page,text
0,1,国内拠点（本店・支店・営業部・出張所）のご案内\n＊2月曜日の0：00から7：00まではご利...
1,2,国内拠点（本店・支店・営業部・出張所）のご案内\n＊2月曜日の0：00から7：00まではご利...
2,3,国内拠点（本店・支店・営業部・出張所）のご案内\n＊2月曜日の0：00から7：00まではご利...
3,4,国内拠点（本店・支店・営業部・出張所）のご案内\n＊2月曜日の0：00から7：00まではご利...
4,5,国内拠点（本店・支店・営業部・出張所）のご案内\n＊2月曜日の0：00から7：00まではご利...


In [3]:
df.shape[0]

7

In [4]:
len(df)

7

In [5]:
from sentence_transformers import SentenceTransformer
import numpy as np

# モデルの読み込み（軽量で日本語対応）
model = SentenceTransformer("intfloat/multilingual-e5-small")

# チャンクからテキストだけ抽出
texts = [chunk["text"] for chunk in all_chunks]

# ベクトル化（正規化あり）
embeddings = model.encode(texts, normalize_embeddings=True)

# ベクトルを各チャンクに追加（NumPy形式）
for chunk, emb in zip(all_chunks, embeddings):
    chunk["embedding"] = emb


/opt/anaconda3/envs/search-vector-semantic-files/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# チャンク数とベクトル次元の確認
print(f"チャンク数: {len(all_chunks)}")
print(f"ベクトル次元: {len(all_chunks[0]['embedding'])}")

# 最初の1件を確認
print("\n最初のチャンクのベクトル（先頭5次元）:")
print(all_chunks[0]["embedding"][:5])


チャンク数: 54
ベクトル次元: 384

最初のチャンクのベクトル（先頭5次元）:
[ 0.04392845 -0.01558716 -0.02172278  0.00289719  0.04273948]


In [7]:
import faiss
import numpy as np
import pickle
from pathlib import Path
from glob import glob
import pdfplumber
from sentence_transformers import SentenceTransformer

pdf_dir = Path("../data/input/pdf")
pdf_paths = sorted(pdf_dir.glob("*.pdf"))

all_chunks = []

# モデルの準備（日本語対応、軽量）
model = SentenceTransformer("intfloat/multilingual-e5-small")

# 各PDFを処理
for pdf_path in pdf_paths:
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            if text:
                all_chunks.append({
                    "chunk_id": f"{pdf_path}_page_{i+1}",
                    "text": text.strip(),
                    "page": i + 1,
                    "file_path": str(pdf_path)
                })

print(f"総チャンク数: {len(all_chunks)}")

# ベクトル化
texts = [chunk["text"] for chunk in all_chunks]
embeddings = model.encode(texts, normalize_embeddings=True)
for chunk, emb in zip(all_chunks, embeddings):
    chunk["embedding"] = emb

# FAISSインデックス構築・登録
dim = len(all_chunks[0]["embedding"])
index = faiss.IndexFlatL2(dim)
vecs = np.array([chunk["embedding"] for chunk in all_chunks]).astype("float32")
index.add(vecs)
print(f"FAISSインデックスに登録済み: {index.ntotal} 件")

index_path = Path("../data/faiss/faiss.index")
meta_path = Path("../data/faiss/faiss_meta.pkl")
index_path.parent.mkdir(parents=True, exist_ok=True)

faiss.write_index(index, str(index_path))
print(f"インデックス保存: {index_path}")

for chunk in all_chunks:
    chunk["embedding"] = None  # 軽量化

with open(meta_path, "wb") as f:
    pickle.dump(all_chunks, f)
print(f"メタ情報保存: {meta_path}")


総チャンク数: 54
FAISSインデックスに登録済み: 54 件
インデックス保存: ../data/faiss/faiss.index
メタ情報保存: ../data/faiss/faiss_meta.pkl


In [9]:
import faiss
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer
import pandas as pd

index = faiss.read_index("../data/faiss/faiss.index")

with open("../data/faiss/faiss_meta.pkl", "rb") as f:
    chunks = pickle.load(f)

model = SentenceTransformer("intfloat/multilingual-e5-small")

query = "ラオスについて"

# クエリをベクトル化（正規化あり）
query_vec = model.encode([query], normalize_embeddings=True).astype("float32")

# FAISS検索（上位5件）
k = 5
D, I = index.search(query_vec, k)  # D: 距離、I: インデックス

# 検索結果の整形
results = []
for idx, dist in zip(I[0], D[0]):
    if idx < len(chunks):
        result = {
            "page": chunks[idx]["page"],
            "file_path": chunks[idx]["file_path"],
            "text": chunks[idx]["text"][:150] + "...",  # テキスト冒頭だけ表示
            "score(L2距離)": float(dist)
        }
        results.append(result)

df = pd.DataFrame(results)
df


,page,file_path,text,score(L2距離)
0,1,../data/input/pdf/news130711.pdf,2013年7月11日\n株式会社みずほ銀行\nラオス計画投資省との業務協力覚書の締結について...,0.261698
1,16,../data/input/pdf/tenpo_all.pdf,国内拠点（本店・支店・営業部・出張所）のご案内\n＊2月曜日の0：00から7：00まではご利...,0.353209
2,1,../data/input/pdf/20250328_4release_jp.pdf,2025年3月28日\n株式会社みずほフィナンシャルグループ\n株式会社みずほ銀行\nみずほ...,0.353838
3,1,../data/input/pdf/news131119.pdf,2013年11月19日\n株式会社みずほ銀行\nアリババグループとの業務協力覚書の締結につい...,0.357040
4,3,../data/input/pdf/20240220release_jp.pdf,(4) 主な事業内容 金属資源、エネルギー、プロジェクト、モビリティ、化学品、鉄鋼\n製品、...,0.357286
